
## Compare 10 regression model
> 這裡差不多完成了，把我們的 data 加到下面 dataset 應該就能跑了 

* Stochastic Gradient Descent
* k-NN
* SVM with 
    * linear kernel
    * RBF kernel
* Gaussian Procress with
    * RBF kernel
    * Dot Product kernel
* Decision Tree
* Random Forest
* AdaBoost
* Neural Net(Perceptron)

In [1]:

import warnings; warnings.simplefilter('ignore')
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt 

filename = 'pm25_complete.csv'
df = pd.read_csv(filename)
df = df.dropna()
df = df.drop(columns=['SiteName', 'date'])
inputNum = len(df.columns)-3


In [2]:
df.head()

,hour,s_h0,s_t0,s_d0_b1,s_d0_b2,s_d0_b3,s_d0,s_d1,s_d2
3,18,100.0,20.584000,1.857143,1.588235,1.642857,0.300000,0.0,0.0
4,19,100.0,20.575000,0.300000,1.857143,1.588235,2.000000,0.0,0.0
5,20,100.0,20.367368,2.000000,0.300000,1.857143,0.315789,0.0,0.0
6,21,100.0,20.250000,0.315789,2.000000,0.300000,1.923077,0.0,0.0
7,23,100.0,20.397857,1.923077,0.315789,2.000000,5.357143,0.0,0.0


In [3]:
X = df[['hour', 's_h0', 's_t0']].values
Y = df[['s_d0']].values
airdata = (X, Y)

In [4]:
# import pipeline
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import warnings; warnings.simplefilter('ignore')
import numpy as np
import pandas as pd


# Preprocess
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import datasets


# import regressors
# for regressors, score function is R2-score in sklearn.metrics
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor

# import MSE
from sklearn.metrics import mean_squared_error


# Result
col = ['model', 'MSE', 'R2']
regResult = pd.DataFrame(columns=col)


# Regressor List
regressornames = ["SGD", "Nearest Neighbors", #"Linear SVR", 
                  #"RBF SVR", 
         #"RBF Gaussian Process", "DotKernel Gaussian Process",
         "Decision Tree", "Random Forest", "AdaBoost", "Neural Net",
         "Ridge1", "Ridge2", "Ridge3", "Ridge4", "Ridge5"]

regressors = [
    SGDRegressor(max_iter=10000, tol=1e-3),
    KNeighborsRegressor(3), 
    #SVR(kernel='linear', C=1e3),
    #SVR(kernel='rbf', C=1e3, gamma=0.1),
    #GaussianProcessRegressor(kernel=1.0 * RBF(1.0)),
    #GaussianProcessRegressor(kernel=DotProduct() + WhiteKernel()),
    DecisionTreeRegressor(max_depth=5),
    RandomForestRegressor(max_depth=5, n_estimators=10, max_features=2), 
    AdaBoostRegressor(),
    MLPRegressor(hidden_layer_sizes=(5,),
                   learning_rate='adaptive',
                   max_iter=10000,
                   learning_rate_init=0.01,
                   alpha=0.01),
    make_pipeline(PolynomialFeatures(1), Ridge()),
    make_pipeline(PolynomialFeatures(2), Ridge()),
    make_pipeline(PolynomialFeatures(3), Ridge()),
    make_pipeline(PolynomialFeatures(4), Ridge()),
    make_pipeline(PolynomialFeatures(5), Ridge())
    ]

# Dataset List

datasets = [airdata]

# iterate over datasets
for ds_cnt, ds in enumerate(datasets):
    X, y = ds
    regResult = pd.DataFrame(columns=col)
    
    # Standardize
    X = StandardScaler().fit_transform(X)
    
    # Data Split
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.3, random_state=42)

    # iterate over regressors
    for name, regressor in zip(regressornames, regressors):
        regressor.fit(X_train, y_train)
        score = regressor.score(X_test, y_test)
        y_pred = regressor.predict(X_test)
        print(name, "score: ", score)
        regResult = regResult.append({'model': name, 
                          'MSE': mean_squared_error(y_test, y_pred), 
                          'R2': score}, 
                          ignore_index=True)        
    display(regResult)
    print('-'*40)
    

SGD score:  0.011176323587041459
Nearest Neighbors score:  -0.1266302470255749
Decision Tree score:  0.08527322154965467
Random Forest score:  0.09556509722326512
AdaBoost score:  -0.011640323201435265
Neural Net score:  0.03374064543080868
Ridge1 score:  0.012018110742621602
Ridge2 score:  0.02718915382505693
Ridge3 score:  0.039591137884138816
Ridge4 score:  0.04767690182757811
Ridge5 score:  0.062160423316576896


,model,MSE,R2
0,SGD,416.134126,0.011176
1,Nearest Neighbors,474.128305,-0.126630
2,Decision Tree,384.951370,0.085273
3,Random Forest,380.620162,0.095565
4,AdaBoost,425.736228,-0.011640
5,Neural Net,406.638213,0.033741
6,Ridge1,415.779871,0.012018
7,Ridge2,409.395326,0.027189
8,Ridge3,404.176106,0.039591
9,Ridge4,400.773313,0.047677


----------------------------------------
